# Voting Schemes

In this project, four different voting schemes will be analysed using the TVA:

In [4]:
import numpy as np
from abc import ABC, abstractmethod

In [5]:
class VotingScheme(ABC):
    def __init__(self, mapping=None):
        # TODO: generate if None
        self.mapping = mapping
    @abstractmethod
    def compute_res(self, candidate_list, preferences)->list:
        raise NotImplemented
        
    def _translate_to_int_array(self, ranking, mapping):
        numbers = []
        for element in ranking:
            index = 1
            for m in mapping:
                if(element == m):
                    numbers.append(index)
                    break
                index += 1
        return np.asarray(numbers)

    def _translate_to_content_array(self, ranking, mapping):
        values = []
        for element in ranking:
            values.append(mapping[element-1])
        return np.asarray(values)

## 1. Voting for one

### Definition

Voters can vote for one candidate. The candidate with the most votes wins.

### Voting Vector

\{1, 0, ..., 0\}

### Implementation

In [9]:
class VotingForOne(VotingScheme):
    def compute_res(candidate_list, preferences):
        votes = self._translate_to_int_array(preferences)
        indexes = np.argmax(votes)
        c = candidate_list[indexes]
        (unique, counts) = np.unique(c, return_counts=True)
        frequencies = np.asarray((unique, counts), dtypes=[('candidate', int), ('votes', int)]).T
        sorted_freqs = np.sort(frequencies, order='votes')
        return self._translate_to_content_array(sorted_freqs[:, 0].T)
        

## 2. Voting for two

### Definition

Voters can vote for two candidates. The two candidates with the most votes win.

### Voting Vector

\{1, 1, 0, ..., 0\}

### Implementation

In [8]:
class VotingForTwo(VotingScheme):
    def compute_res(self, candidate_list, preferences)->list:
        res = defaultdict(int)
        for i in range(2):
            for j in range(preferences.shape[1]):
                res[preferences[i, j]]+=1
        sorted_res=[x[0] for x in sorted(res.items(), key=lambda x: x[1], reverse=True)]
        return sorted_res

## 3. Anti-plurality voting (veto)

### Definition

In anti-plurality voting, the individual voter decides on a certain option it does not want to elect, thus votes for everyone but that one.

### Voting Vector

\{1,1,...,0\}

### Implementation

In [10]:
class AntiPluralityVoting(VotingScheme):
    def compute_res(self, candidate_list, preferences)->list:
        res = defaultdict(int)
        for i in range(preferences.shape[0]-1):
            for j in range(preferences.shape[1]):
                res[preferences[i, j]]+=1
        sorted_res=[x[0] for x in sorted(res.items(), key=lambda x: x[1], reverse=True)]
        return sorted_res

## 4. Borda voting

### Definition

Borda voting requires the voters to submit a full ranking of all candidates (= preference list).

### Voting Vector

\{𝑚−1,𝑚−2,...,1,0\} (where m is the number of alternatives)

### Implementation

In [6]:
# same as in risk_function.ipynb